In [ ]:
import pandas as pd

In [ ]:
# Import data
gs = pd.read_excel('scholar_through_publishorperish.xlsx')
scopus = pd.read_excel('scopus.xlsx')
wos = pd.read_excel('wos.xls')

print(gs.shape, scopus.shape, wos.shape)

Check missing DOIs:

In [ ]:
print(gs.DOI.isna().sum())
print(scopus.DOI.isna().sum())
print(wos.DOI.isna().sum())

Check missing Titles:

In [ ]:
print(gs.Title.isna().sum())
print(scopus.Title.isna().sum())
print(wos['Article Title'].isna().sum())

To get duplicates we will look at the tile, since there are many missing DOIs, especially from Google Scholar.

1. Duplicates within each search engine

In [ ]:
# Find duplicates within each search engine:
duplicates_gs=gs[gs.Title.duplicated(keep='first')].index
duplicates_scopus=scopus[scopus.Title.duplicated(keep='first')].index
duplicates_wos=wos[wos['Article Title'].duplicated(keep='first')].index
# Check number of duplicates:
print('Duplicates in Google Scholar search:', duplicates_gs.shape[0])
print('Duplicates in Scopus search:', duplicates_scopus.shape[0])
print('Duplicates in Web of Science search:', duplicates_wos.shape[0])
# Update tables:
gs.drop(index=duplicates_gs,inplace=True)
scopus.drop(index=duplicates_scopus,inplace=True)
wos.drop(index=duplicates_wos,inplace=True)
# Check new dimensions:
print('Number of results from Google Scholar after removed duplicates:', gs.shape[0])
print('Number of results from Scopus after removed duplicates:', scopus.shape[0])
print('Number of results from Web of Science after removed duplicates:', wos.shape[0])

2. Duplicates among all search engines

In [ ]:
all_titles=pd.Series(pd.concat([gs.Title, scopus.Title, wos['Article Title']]))
duplicated_titles=all_titles[all_titles.duplicated(keep='first')]
print('Number of duplicates across all results:',duplicated_titles.shape[0])

In [ ]:
print("In Google Scholar:",gs.Title.isin(duplicated_titles).sum())
print("In Scopus:",scopus.Title.isin(duplicated_titles).sum())
print("In Web of Science:",wos['Article Title'].isin(duplicated_titles).sum())

    2.1. Keep all the common duplicates in the Google Scholar table and remove these from the Scopus and Web of Science tables.

In [ ]:
common_gs = gs.Title[gs.Title.isin(duplicated_titles)]
scopus=scopus[~scopus.Title.isin(common_gs)]
wos=wos[~wos['Article Title'].isin(common_gs)]

Check that the number of duplicates has changed and that there are none left in the Google Scholar table.

In [ ]:
all_titles_2=pd.Series(pd.concat([gs.Title, scopus.Title, wos['Article Title']]))
duplicated_titles_2=all_titles_2[all_titles_2.duplicated(keep='first')]
print('Remaining number of duplicates:',duplicated_titles_2.shape[0])

In [ ]:
print('Number of duplicates remaining in Google Scholar:', gs.Title.isin(duplicated_titles_2).sum())
print('Number of duplicates remaining in Scopus:', scopus.Title.isin(duplicated_titles_2).sum())
print('Number of duplicates remaining in Web of Science:', wos['Article Title'].isin(duplicated_titles_2).sum())

    2.2. Keep the remaining common duplicates in the Scopus table and remove these from the Web of Science table.

In [ ]:
common_scopus = scopus.Title[scopus.Title.isin(duplicated_titles_2)]

In [ ]:
wos=wos[~wos['Article Title'].isin(common_scopus)]

Check there are not any duplicates left.

In [ ]:
all_titles_3=pd.Series(pd.concat([gs.Title, scopus.Title, wos['Article Title']]))
duplicated_titles_3=all_titles_3[all_titles_3.duplicated(keep='first')]
print('Remaining number of duplicates:',duplicated_titles_3.shape[0])

In [ ]:
# Check final shape of tables:
print("Google Scholar:", gs.shape)
print("Scopus:", scopus.shape)
print("Web of Science:", wos.shape)
print("Total number of unique results:",gs.shape[0]+scopus.shape[0]+wos.shape[0])

Save results.

In [ ]:
gs.to_excel('scholar_unique.xlsx',index=0)
scopus.to_excel('scopus_unique.xlsx',index=0)
wos.to_excel('wos_unique.xlsx',index=0)